In [1]:
from bs4 import BeautifulSoup, Comment
import requests
import pandas as pd
import json
import re
import getpass
import psycopg2
import numpy as np
from psycopg2.extensions import adapt, register_adapter, AsIs
import sys
from IPython.core.display import HTML
import random
import time
import csv

In [2]:
!{sys.executable} -m pip install pip --upgrade
!{sys.executable} -m pip install nfl_data_py
!{sys.executable} -m pip install pyarrow
!{sys.executable} -m pip install fastparquet

     |████████████████████████████████| 2.1MB 3.4MB/s eta 0:00:01
  Found existing installation: pip 19.2.2
    Uninstalling pip-19.2.2:
      Successfully uninstalled pip-19.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 30.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 29.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.4 MB/s eta 0:00:00
  Created wheel for nfl_data_py: filename=nfl_data_py-0.3.0-py3-none-any.whl size=25089 sha256=772747e466c0e2fda1bee7c024f3e655becf4cb5168ccee26f16197e5bb59bef
  Stored in d

In [ ]:
# You may need to restart the kernerl here:
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
from bs4 import BeautifulSoup, Comment
import requests
import pandas as pd
import json
import re
import getpass
import psycopg2
import numpy as np
from psycopg2.extensions import adapt, register_adapter, AsIs
import sys
from IPython.core.display import HTML
import random
import time
import csv
import nfl_data_py as nfl

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
mypasswd = getpass.getpass()
mysso = 'mtrkkf'
dbname = 'capstonesp23olt05'
schema = 'team05'

········


In [3]:
connection = psycopg2.connect(database = dbname, user = mysso, host = 'pgsql.dsa.lan', password = mypasswd)

In [4]:
years = [2015,2016,2017,2018,2019,2020,2021,2022]

In [5]:
randomizer = [2, 3, 4]

In [6]:
url = 'https://www.pro-football-reference.com/boxscores/'

## Team Table:

In [7]:
team_df = nfl.import_team_desc()
team_df = team_df[team_df.team_abbr.isin(['STL','SD','LAR', 'OAK']) == False]

pfr_abbr_mapping = { 
    'name':['Tampa Bay Buccaneers', 'Philadelphia Eagles', 'Atlanta Falcons',  'New England Patriots', 
            'New York Jets', 'Chicago Bears', 'Green Bay Packers', 'Buffalo Bills', 
            'Miami Dolphins', 'Minnesota Vikings', 'Cincinnati Bengals', 'Kansas City Chiefs', 
            'Seattle Seahawks', 'Jacksonville Jaguars', 'New York Giants', 'Tennessee Titans', 
            'Pittsburgh Steelers', 'Carolina Panthers', 'Arizona Cardinals', 'Indianapolis Colts', 
            'Denver Broncos', 'Detroit Lions', 'Houston Texans', 'New Orleans Saints', 'Cleveland Browns', 
            'Baltimore Ravens', 'Washington Commanders', 'San Francisco 49ers', 'Dallas Cowboys',
            'Los Angeles Rams', 'Las Vegas Raiders', 'Los Angeles Chargers'], 
    'pfr_abbreviation':['TAM', 'PHI', 'ATL', 'NWE', 'NYJ', 'CHI', 'GNB', 'BUF', 'MIA', 'MIN', 'CIN', 'KAN', 
        'SEA', 'JAX', 'NYG', 'OTI', 'PIT', 'CAR', 'CRD', 'CLT', 'DEN', 'DET', 'HTX', 'NOR', 'CLE', 'RAV', 'WAS', 
        'SFO', 'DAL', 'RAM', 'RAI', 'SDG']
}
pfr_abbr_mapping = pd.DataFrame(pfr_abbr_mapping)

team_df = pd.merge(team_df, pfr_abbr_mapping, left_on = ['team_name'], right_on = ['name'], how = 'left')

team_df = team_df[["team_id", "team_name", "team_abbr", "pfr_abbreviation", "team_nick", "team_conf", "team_division"]]


In [8]:
create_statement = f"""
DROP TABLE IF EXISTS team;

CREATE TABLE team (
    team_id            INT PRIMARY KEY NOT NULL,
    team_name          VARCHAR(100),
    team_abbreviation  VARCHAR(10),
    pfr_abbreviation   VARCHAR(10),
    team_nickname      VARCHAR(50),
    team_conference    VARCHAR(10),
    team_division      VARCHAR(50)
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [9]:
insert_statement = f"""
INSERT INTO team (team_id, team_name, team_abbreviation, pfr_abbreviation, team_nickname, team_conference, team_division)
VALUES (%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in team_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [10]:
select_statement = f"""
SELECT * 
FROM team;
"""

sql_team_df = pd.read_sql_query(select_statement, con = connection)
sql_team_df.head(3)

,team_id,team_name,team_abbreviation,pfr_abbreviation,team_nickname,team_conference,team_division
0,3800,Arizona Cardinals,ARI,CRD,Cardinals,NFC,NFC West
1,200,Atlanta Falcons,ATL,ATL,Falcons,NFC,NFC South
2,325,Baltimore Ravens,BAL,RAV,Ravens,AFC,AFC North


## Game Table:

In [11]:
games_df = nfl.import_schedules(years)

games_df['home_team'] = games_df['home_team'].replace(to_replace='STL', value='LA')
games_df['home_team'] = games_df['home_team'].replace(to_replace='SD', value='LAC')
games_df['home_team'] = games_df['home_team'].replace(to_replace='OAK', value='LV')
games_df['away_team'] = games_df['away_team'].replace(to_replace='STL', value='LA')
games_df['away_team'] = games_df['away_team'].replace(to_replace='SD', value='LAC')
games_df['away_team'] = games_df['away_team'].replace(to_replace='OAK', value='LV')

home_team_df = team_df[["team_id", "team_abbr"]]
away_team_df = team_df[["team_id", "team_abbr"]]

home_team_df.columns.values[0] = 'home_team_id'
home_team_df.columns.values[1] = 'home_team_abv'

away_team_df.columns.values[0] = 'away_team_id'
away_team_df.columns.values[1] = 'away_team_abv'

games_df = pd.merge(games_df, home_team_df, left_on = ['home_team'], right_on = ['home_team_abv'], how = 'inner')
games_df = pd.merge(games_df, away_team_df, left_on = ['away_team'], right_on = ['away_team_abv'], how = 'inner')

games_df['season'] = games_df['game_id'].str[0:4]


games_df['point_differential'] = abs(games_df['away_score'] - games_df['home_score'])
games_df['home_point_differential'] = games_df['home_score'] - games_df['away_score']
games_df['away_point_differential'] = games_df['away_score'] - games_df['home_score']

games_df = games_df[[
    'game_id', 'game_type', 'div_game', 'season', 'week', 'gameday', 'weekday', 'gametime',  
    'away_team_id', 'home_team_id', 'away_score', 'home_score', 'away_rest', 'home_rest', 'overtime',   
    'away_qb_id', 'home_qb_id', 'away_coach', 'home_coach', 'roof', 'surface', 'temp', 'wind',
    'point_differential', 'home_point_differential', 'away_point_differential'
]]


In [12]:
create_statement = f"""
DROP TABLE IF EXISTS game;

CREATE TABLE game (
    game_id                 VARCHAR(50) PRIMARY KEY NOT NULL,
    game_type               VARCHAR(50),
    game_divisional         INT,
    gametime_year           INT,
    gametime_week           INT,
    gametime_day            DATE,
    gametime_weekday        VARCHAR(50),
    gametime_time           TIME,
    away_team_id            INT,
    home_team_id            INT,
    away_team_score         DECIMAL,
    home_team_score         DECIMAL,
    away_team_rest_days     INT,
    home_team_rest_days     INT,
    overtime_played         DECIMAL,
    away_team_qb_id         VARCHAR(50),
    home_team_qb_id         VARCHAR(50),
    away_team_coach         VARCHAR(100),
    home_team_coach         VARCHAR(100),
    conditions_roof         VARCHAR(50),
    conditions_surface      VARCHAR(50),
    conditions_temp         DECIMAL,
    conditions_wind         DECIMAL,
    point_differential      INT,
    home_point_differential INT,
    away_point_differential INT
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [13]:
insert_statement = f"""
INSERT INTO game (game_id, game_type, game_divisional, gametime_year, gametime_week, gametime_day, 
    gametime_weekday, gametime_time, away_team_id, home_team_id, away_team_score, home_team_score, 
    away_team_rest_days, home_team_rest_days, overtime_played, away_team_qb_id, home_team_qb_id, 
    away_team_coach, home_team_coach, conditions_roof, conditions_surface, conditions_temp, conditions_wind,
    point_differential, home_point_differential, away_point_differential)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in games_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [14]:
select_statement = f"""
SELECT *
FROM game;
"""

sql_game_df = pd.read_sql_query(select_statement, con = connection)
sql_game_df.head(3)

,game_id,game_type,game_divisional,gametime_year,gametime_week,gametime_day,gametime_weekday,gametime_time,away_team_id,home_team_id,...,home_team_qb_id,away_team_coach,home_team_coach,conditions_roof,conditions_surface,conditions_temp,conditions_wind,point_differential,home_point_differential,away_point_differential
0,2015_01_PIT_NE,REG,0,2015,1,2015-09-10,Thursday,20:30:00,3900,3200,...,00-0019596,Mike Tomlin,Bill Belichick,outdoors,fieldturf,65.0,7.0,7,7,-7
1,2016_20_PIT_NE,CON,0,2016,20,2017-01-22,Sunday,18:40:00,3900,3200,...,00-0019596,Mike Tomlin,Bill Belichick,outdoors,fieldturf,41.0,10.0,19,19,-19
2,2019_01_PIT_NE,REG,0,2019,1,2019-09-08,Sunday,20:20:00,3900,3200,...,00-0019596,Mike Tomlin,Bill Belichick,outdoors,grass,66.0,2.0,30,30,-30


## Lines Table:

In [15]:
lines_df = pd.read_csv('/dsa/groups/capstonesp2023/online/Team05/NFL_betting_Full.csv', sep=',')

lines_df['home_team'] = lines_df['home_team'].replace(to_replace='St. Louis Rams', value='Los Angeles Rams')
lines_df['home_team'] = lines_df['home_team'].replace(to_replace='San Diego Chargers', value='Los Angeles Chargers')
lines_df['home_team'] = lines_df['home_team'].replace(to_replace='Oakland Raiders', value='Las Vegas Raiders')
lines_df['home_team'] = lines_df['home_team'].replace(to_replace='Washington Redskins', value='Washington Commanders')
lines_df['home_team'] = lines_df['home_team'].replace(to_replace='Washington Football Team', value='Washington Commanders')

lines_df['away_team'] = lines_df['away_team'].replace(to_replace='St. Louis Rams', value='Los Angeles Rams')
lines_df['away_team'] = lines_df['away_team'].replace(to_replace='San Diego Chargers', value='Los Angeles Chargers')
lines_df['away_team'] = lines_df['away_team'].replace(to_replace='Oakland Raiders', value='Las Vegas Raiders')
lines_df['away_team'] = lines_df['away_team'].replace(to_replace='Washington Redskins', value='Washington Commanders')
lines_df['away_team'] = lines_df['away_team'].replace(to_replace='Washington Football Team', value='Washington Commanders')

home_team_df = team_df[["team_id", "team_name"]]
away_team_df = team_df[["team_id", "team_name"]]

home_team_df.columns.values[0] = 'home_team_id'
home_team_df.columns.values[1] = 'home_team_name'

away_team_df.columns.values[0] = 'away_team_id'
away_team_df.columns.values[1] = 'away_team_name'

lines_df = pd.merge(lines_df, home_team_df, left_on = ['home_team'], right_on = ['home_team_name'], how = 'inner')
lines_df = pd.merge(lines_df, away_team_df, left_on = ['away_team'], right_on = ['away_team_name'], how = 'inner')

line_games_df = games_df[['game_id', 'gameday', 'away_team_id', 'home_team_id']]

lines_df = pd.merge(lines_df, line_games_df, left_on = ['home_team_id', 'away_team_id', 'date'], right_on = ['home_team_id', 'away_team_id', 'gameday'], how = 'inner')

lines_df = lines_df[['game_id', 'home_line_open', 'home_line_min', 'home_line_max', 'home_line_close',
    'away_line_open', 'away_line_min', 'away_line_max', 'away_line_close', 'home_line_odds_open', 
    'home_line_odds_min', 'home_line_odds_max', 'home_line_odds_close', 'away_line_odds_open', 
    'away_line_odds_min', 'away_line_odds_max', 'away_line_odds_close'
]]

lines_df['home_line_movement'] = lines_df['home_line_open'] - lines_df['home_line_close']
lines_df['away_line_movement'] = lines_df['away_line_open'] - lines_df['away_line_close']

lines_df['home_line_odds_movement'] = lines_df['home_line_odds_open'] - lines_df['home_line_odds_close']
lines_df['away_line_odds_movement'] = lines_df['away_line_odds_open'] - lines_df['away_line_odds_close']

print("Home Team Line Movement 10% Percentile: ", str(np.percentile(lines_df.home_line_movement, 10)))
print("Home Team Line Movement 25% Percentile: ", str(np.percentile(lines_df.home_line_movement, 25)))
print("Home Team Line Movement 75% Percentile: ", str(np.percentile(lines_df.home_line_movement, 75)))
print("Home Team Line Movement 90% Percentile: ", str(np.percentile(lines_df.home_line_movement, 90)))
print('\n')
print("Away Team Line Movement 10% Percentile: ", str(np.percentile(lines_df.away_line_movement, 10)))
print("Away Team Line Movement 25% Percentile: ", str(np.percentile(lines_df.away_line_movement, 25)))
print("Away Team Line Movement 75% Percentile: ", str(np.percentile(lines_df.away_line_movement, 75)))
print("Away Team Line Movement 90% Percentile: ", str(np.percentile(lines_df.away_line_movement, 90)))

lines_df.loc[lines_df['home_line_movement'] <= -2, 'home_line_movement_cohort'] = 'Heavy Movement to Home'
lines_df.loc[(lines_df['home_line_movement'] > -2) & (lines_df['home_line_movement'] <= -1), 'home_line_movement_cohort'] = 'Slight Movement to Home'  
lines_df.loc[lines_df['home_line_movement'] >= 2, 'home_line_movement_cohort'] = 'Heavy Movement to Away'
lines_df.loc[(lines_df['home_line_movement'] < 2) & (lines_df['home_line_movement'] >= 1), 'home_line_movement_cohort'] = 'Slight Movement to Away'  
lines_df.loc[(lines_df['home_line_movement'] > -1) & (lines_df['home_line_movement'] < 1), 'home_line_movement_cohort'] = 'Little to No Movement'

# lines_df.loc[lines_df['away_line_movement'] <= -2, 'away_line_movement_cohort'] = 'Heavy Movement to Favored'
# lines_df.loc[(lines_df['away_line_movement'] > -2) & (lines_df['away_line_movement'] <= -.5), 'away_line_movement_cohort'] = 'Slight Movement to Favored'  
# lines_df.loc[lines_df['away_line_movement'] >= 2, 'away_line_movement_cohort'] = 'Heavy Movement to Unfavored'
# lines_df.loc[(lines_df['away_line_movement'] < 2) & (lines_df['away_line_movement'] >= 1), 'away_line_movement_cohort'] = 'Slight Movement to Unfavored'  
# lines_df.loc[(lines_df['away_line_movement'] > -.5) & (lines_df['away_line_movement'] < 1), 'away_line_movement_cohort'] = 'Little to No Movement'


Home Team Line Movement 10% Percentile:  -2.0
Home Team Line Movement 25% Percentile:  -1.0
Home Team Line Movement 75% Percentile:  0.5
Home Team Line Movement 90% Percentile:  2.0


Away Team Line Movement 10% Percentile:  -2.0
Away Team Line Movement 25% Percentile:  -0.5
Away Team Line Movement 75% Percentile:  1.0
Away Team Line Movement 90% Percentile:  2.0


In [16]:
create_statement = f"""
DROP TABLE IF EXISTS line;

CREATE TABLE line (
    game_id                   VARCHAR(50) PRIMARY KEY NOT NULL,
    home_line_open            DECIMAL,
    home_line_min             DECIMAL,
    home_line_max             DECIMAL,
    home_line_close           DECIMAL,
    away_line_open            DECIMAL,
    away_line_min             DECIMAL,
    away_line_max             DECIMAL,
    away_line_close           DECIMAL,
    home_line_odds_open       DECIMAL,
    home_line_odds_min        DECIMAL,
    home_line_odds_max        DECIMAL,
    home_line_odds_close      DECIMAL,
    away_line_odds_open       DECIMAL,
    away_line_odds_min        DECIMAL,
    away_line_odds_max        DECIMAL,
    away_line_odds_close      DECIMAL,
    home_line_movement        DECIMAL,
    away_line_movement        DECIMAL,
    home_line_odds_movement   DECIMAL,
    away_line_odds_movement   DECIMAL,
    line_movement_cohort VARCHAR(150)
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [17]:
insert_statement = f"""
INSERT INTO line (game_id, home_line_open, home_line_min, home_line_max, home_line_close, away_line_open,
    away_line_min, away_line_max, away_line_close, home_line_odds_open, home_line_odds_min, home_line_odds_max, 
    home_line_odds_close, away_line_odds_open, away_line_odds_min, away_line_odds_max, away_line_odds_close,
    home_line_movement, away_line_movement, home_line_odds_movement, away_line_odds_movement, 
    line_movement_cohort)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in lines_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [18]:
select_statement = f"""
SELECT *
FROM line;
"""

sql_line_df = pd.read_sql_query(select_statement, con = connection)
sql_line_df.head(3)

,game_id,home_line_open,home_line_min,home_line_max,home_line_close,away_line_open,away_line_min,away_line_max,away_line_close,home_line_odds_open,...,home_line_odds_close,away_line_odds_open,away_line_odds_min,away_line_odds_max,away_line_odds_close,home_line_movement,away_line_movement,home_line_odds_movement,away_line_odds_movement,line_movement_cohort
0,2022_20_DAL_SF,-4.0,-4.0,-3.5,-3.5,4.0,3.5,4.0,3.5,1.91,...,1.91,1.91,1.91,1.91,1.91,-0.5,0.5,0.00,0.00,Little to No Movement
1,2017_07_DAL_SF,6.0,5.5,6.5,6.5,-6.0,-6.5,-5.5,-6.5,1.91,...,1.93,2.00,1.94,1.95,1.97,-0.5,0.5,-0.02,0.03,Little to No Movement
2,2016_04_DAL_SF,3.0,1.0,3.0,1.0,-3.0,-3.0,-1.0,-1.0,1.95,...,2.00,1.95,1.95,1.95,1.91,2.0,-2.0,-0.05,0.04,Heavy Movement to Away


## Player Weekly Stats Table:

In [19]:
player_week_stats_df = nfl.import_weekly_data(years)

player_team_df = team_df[["team_id", "team_abbr"]]

player_week_stats_df = pd.merge(player_week_stats_df, player_team_df, left_on = ['recent_team'], right_on = ['team_abbr'], how = 'inner')

player_games_home_df = games_df[['game_id', 'season', 'week', 'home_team_id']]
player_games_away_df = games_df[['game_id', 'season', 'week', 'away_team_id']]

player_games_home_df['season'] = player_games_home_df['season'].astype(int)
player_games_away_df['season'] = player_games_away_df['season'].astype(int)

player_week_stats_df = pd.merge(player_week_stats_df, player_games_home_df, left_on = ['season', 'week', 'team_id'], right_on = ['season', 'week', 'home_team_id'], how = 'left')
player_week_stats_df = pd.merge(player_week_stats_df, player_games_away_df, left_on = ['season', 'week', 'team_id'], right_on = ['season', 'week', 'away_team_id'], how = 'left')

player_week_stats_df['game_id'] = player_week_stats_df[['game_id_x', 'game_id_y']].bfill(axis=1).iloc[:, 0]

player_week_stats_df = player_week_stats_df[[
    'player_id', 'team_id', 'game_id', 'player_display_name', 'position_group', 
    'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards', 
    'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 
    'passing_epa', 'passing_2pt_conversions', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
    'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 
    'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 
    'receiving_2pt_conversions', 'special_teams_tds'
]]

Downcasting floats.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [20]:
create_statement = f"""
DROP TABLE IF EXISTS player_weekly_stats;

CREATE TABLE player_weekly_stats (
    player_id                    VARCHAR(50) NOT NULL,
    team_id                      INT,
    game_id                      VARCHAR(100),
    player_name                  VARCHAR(150),
    position                     VARCHAR(10),  
    completions                  INT,
    attempts                     INT,
    passing_yards                DECIMAL,
    passing_tds                  INT,
    interceptions                DECIMAL,
    sacks                        DECIMAL,
    sack_yards                   DECIMAL,
    sack_fumbles                 INT,
    sack_fumbles_lost            INT,
    passing_air_yards            DECIMAL,
    passing_yards_after_catch    DECIMAL,
    passing_first_downs          DECIMAL,
    passing_epa                  DECIMAL,
    passing_2pt_conversions      INT,
    carries                      INT,
    rushing_yards                DECIMAL,
    rushing_tds                  INT,
    rushing_fumbles              DECIMAL,
    rushing_fumbles_lost         DECIMAL,
    rushing_first_downs          DECIMAL,
    rushing_epa                  DECIMAL,
    rushing_2pt_conversions      INT,
    receptions                   INT,
    targets                      INT,
    receiving_yards              DECIMAL,
    receiving_tds                INT,
    receiving_fumbles            DECIMAL,
    receiving_fumbles_lost       DECIMAL,
    receiving_air_yards          DECIMAL,
    receiving_yards_after_catch  DECIMAL,
    receiving_first_downs        DECIMAL,
    receiving_epa                DECIMAL,
    receiving_2pt_conversions    INT,
    special_teams_tds            DECIMAL
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [21]:
insert_statement = f"""
INSERT INTO player_weekly_stats (player_id, team_id, game_id, player_name, position, completions, attempts, 
    passing_yards, passing_tds, interceptions, sacks, sack_yards, sack_fumbles, sack_fumbles_lost, 
    passing_air_yards, passing_yards_after_catch, passing_first_downs, passing_epa, passing_2pt_conversions, 
    carries, rushing_yards, rushing_tds, rushing_fumbles, rushing_fumbles_lost, rushing_first_downs, rushing_epa, 
    rushing_2pt_conversions, receptions, targets, receiving_yards, receiving_tds, receiving_fumbles, 
    receiving_fumbles_lost, receiving_air_yards, receiving_yards_after_catch, receiving_first_downs, 
    receiving_epa, receiving_2pt_conversions, special_teams_tds)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in player_week_stats_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [22]:
select_statement = f"""
SELECT *
FROM player_weekly_stats;
"""

sql_player_weekly_stats_df = pd.read_sql_query(select_statement, con = connection)
sql_player_weekly_stats_df.head(3)

,player_id,team_id,game_id,player_name,position,completions,attempts,passing_yards,passing_tds,interceptions,...,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,special_teams_tds
0,00-0007091,2200,2015_04_JAX_IND,Matt Hasselbeck,QB,30,47,282.0,1,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,0.0
1,00-0007091,2200,2015_05_IND_HOU,Matt Hasselbeck,QB,18,29,213.0,2,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,0.0
2,00-0007091,2200,2015_11_IND_ATL,Matt Hasselbeck,QB,23,32,213.0,2,2.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,0.0


## Player Season Stats Table:

In [23]:
yearly_player_stats_df = nfl.import_seasonal_data(years)

yearly_player_stats_df = yearly_player_stats_df[[
    'player_id', 'season', 'games', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 
    'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 
    'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'carries', 'rushing_yards', 'rushing_tds', 
    'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
    'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 
    'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'special_teams_tds'
]]

In [24]:
create_statement = f"""
DROP TABLE IF EXISTS player_yearly_stats;

CREATE TABLE player_yearly_stats (
    player_id                    VARCHAR(50) NOT NULL,
    year                         INT,
    games                        INT,
    completions                  INT,
    attempts                     INT,
    passing_yards                DECIMAL,
    passing_tds                  INT,
    interceptions                DECIMAL,
    sacks                        DECIMAL,
    sack_yards                   DECIMAL,
    sack_fumbles                 INT,
    sack_fumbles_lost            INT,
    passing_air_yards            DECIMAL,
    passing_yards_after_catch    DECIMAL,
    passing_first_downs          DECIMAL,
    passing_epa                  DECIMAL,
    passing_2pt_conversions      INT,
    carries                      INT,
    rushing_yards                DECIMAL,
    rushing_tds                  INT,
    rushing_fumbles              DECIMAL,
    rushing_fumbles_lost         DECIMAL,
    rushing_first_downs          DECIMAL,
    rushing_epa                  DECIMAL,
    rushing_2pt_conversions      INT,
    receptions                   INT,
    targets                      INT,
    receiving_yards              DECIMAL,
    receiving_tds                INT,
    receiving_fumbles            DECIMAL,
    receiving_fumbles_lost       DECIMAL,
    receiving_air_yards          DECIMAL,
    receiving_yards_after_catch  DECIMAL,
    receiving_first_downs        DECIMAL,
    receiving_epa                DECIMAL,
    receiving_2pt_conversions    INT,
    special_teams_tds            DECIMAL
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [25]:
insert_statement = f"""
INSERT INTO player_yearly_stats (player_id, year, games, completions, attempts, passing_yards, passing_tds, 
    interceptions, sacks, sack_yards, sack_fumbles, sack_fumbles_lost, passing_air_yards, passing_yards_after_catch, 
    passing_first_downs, passing_epa, passing_2pt_conversions, carries, rushing_yards, rushing_tds, 
    rushing_fumbles, rushing_fumbles_lost, rushing_first_downs, rushing_epa, rushing_2pt_conversions,
    receptions, targets, receiving_yards, receiving_tds, receiving_fumbles, receiving_fumbles_lost, 
    receiving_air_yards, receiving_yards_after_catch, receiving_first_downs, receiving_epa,
    receiving_2pt_conversions, special_teams_tds)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in yearly_player_stats_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [26]:
select_statement = f"""
SELECT *
FROM player_yearly_stats;
"""

sql_player_yearly_stats_df = pd.read_sql_query(select_statement, con = connection)
sql_player_yearly_stats_df.head(3)

,player_id,year,games,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,special_teams_tds
0,00-0007091,2015,8,156,256,1690.0,9,5.0,16.0,101.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,00-0010346,2015,10,198,331,2249.0,9,17.0,16.0,95.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,00-0018227,2015,1,0,0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


## Team Weekly Stats Table (depricated):

In [27]:
# create_statement = f"""
# DROP TABLE IF EXISTS team_weekly_stats;

# CREATE TABLE team_weekly_stats (
#     team_name                    VARCHAR(200) NOT NULL, 
#     gametime_date                DATE,
#     first_downs                  INT,          
#     rush_attempts                INT, 
#     rush_yards                   INT, 
#     rush_tds                     INT, 
#     completions                  INT, 
#     completion_attempts          INT, 
#     pass_yards                   INT, 
#     pass_tds                     INT, 
#     interceptions                INT,  
#     sacked                       INT,  
#     sacked_yards                 INT,  
#     net_pass_yards               INT, 
#     total_yards                  INT, 
#     fumbles                      INT, 
#     fumbles_lost                 INT, 
#     turn_overs                   INT, 
#     penalties                    INT,  
#     penalty_yards                INT,  
#     third_downs_conversions      INT, 
#     third_downs                  INT, 
#     fourth_down_conversions      INT,  
#     fourth_down_attempts         INT, 
#     time_of_possession           VARCHAR(10)
# );
# """

# with connection, connection.cursor() as cursor:
#     cursor.execute(create_statement)

In [28]:
# select_statement = f"""
# SELECT game.gametime_day, 
#     game.gametime_year,
#     team.pfr_abbreviation
# FROM game
# JOIN team ON game.home_team_id = team_id
# WHERE game.gametime_year = 2022
# """

# df = pd.read_sql_query(select_statement, con = connection)

# df['gametime_day'] = df['gametime_day'].map(str)
# df['pfr_url'] = url +  df['gametime_day'].str[0:4] + df['gametime_day'].str[5:7] + df['gametime_day'].str[8:10] + '0' + df['pfr_abbreviation'] + '.htm'
# links = df.pfr_url.to_numpy()

# for link in links:
#     try:
#         response = requests.get(link, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.49'})
#         soup = BeautifulSoup(response.content, 'html.parser')

#         results = soup.find_all("div",attrs={"id":"content","role":"main"})

#         table = soup.select_one('#all_team_stats').find_next(text=lambda t: isinstance(t, Comment))
#         table = BeautifulSoup(table, 'html.parser')

#         game_info = []
#         home_stats = []
#         away_stats = []

#         # Iterating for team names and date of game
#         for result in results:
#             #game ---------------
#             text = result.find('h1').get_text().strip().replace('Wild Card - ', '')
#             text = re.split(r'-| at ', text)
#             txt_array = []
#             for word in text:
#                 txt_array.append(word.strip())
#             game_info.append(txt_array)
#             df = pd.DataFrame (game_info, columns = ['home_team','away_team','date'])

#             # Iterating for team stats table
#         for tr in table.select('tr'):
#             #stat ---------------
#             tds = [td.get_text(strip=True) for td in tr.select('td')]
#             home_stats.append(tds[:1])
#             away_stats.append(tds[1:])

#         # Creating dataframe, transpose,drop empty row, reset indes, and name columns for home       
#         df_hm_stats = pd.DataFrame (home_stats, columns = ['1']).transpose().drop([0], axis=1).reset_index(drop=True).rename({1:'first_downs',
#             2:'Rush-Yds-TDs',3:'Cmp-Att-Yd-TD-INT', 4:'Sacked-Yards', 5:'net_pass_yds',6:'total_yds', 7:'Fumbles-Lost',
#             8:'turnovers',9:'Penalties-Yards',10:'Third Down Conv.', 11:'Fourth Down Conv.',12:'time_possession'}, axis=1)

#         # Creating dataframe, transpose,drop empty row, reset indes, and name columns for home
#         df_away_stats = pd.DataFrame (away_stats, columns = ['1']).transpose().drop([0], axis=1).reset_index(drop=True).rename({1:'first_downs',
#             2:'Rush-Yds-TDs',3:'Cmp-Att-Yd-TD-INT', 4:'Sacked-Yards', 5:'net_pass_yds',6:'total_yds', 7:'Fumbles-Lost',
#             8:'turnovers',9:'Penalties-Yards',10:'Third Down Conv.', 11:'Fourth Down Conv.',12:'time_possession'}, axis=1)

#         # Adding the team names and date to the dataframes of home and away
#         df_hm_stats['Team'] = df['home_team']
#         df_hm_stats['Date'] = df['date']
#         df_away_stats['Team'] = df['away_team']
#         df_away_stats['Date'] = df['date']

#         # Date time column conversion for home team
#         df_hm_stats['Date']= pd.to_datetime(df_hm_stats['Date'])

#         # Date time column conversion for home team
#         df_away_stats['Date']= pd.to_datetime(df_away_stats['Date'])

#         # Splitting columns for home team
#         df_hm_stats[['rush_atts', 'rush_yds', 'rush_tds']] = df_hm_stats['Rush-Yds-TDs'].str.split('-', expand=True)
#         df_hm_stats[['cmps','cmp_atts', 'pass_yds', 'pass_tds','ints']] = df_hm_stats['Cmp-Att-Yd-TD-INT'].str.split('-', expand=True)
#         df_hm_stats[['sacked', 'sacked_yds']] = df_hm_stats['Sacked-Yards'].str.split('-', expand=True)
#         df_hm_stats[['fumbles', 'fumbles_lost']] = df_hm_stats['Fumbles-Lost'].str.split('-', expand=True)
#         df_hm_stats[['penalties', 'penalty_yds']] = df_hm_stats['Penalties-Yards'].str.split('-', expand=True)
#         df_hm_stats[['third_dwn_convert', 'third_downs']] = df_hm_stats['Third Down Conv.'].str.split('-', expand=True)
#         df_hm_stats[['fourth_dwn_convert', 'fourth_down_atts']] = df_hm_stats['Fourth Down Conv.'].str.split('-', expand=True)

#         # Splitting columns for away team
#         df_away_stats[['rush_atts', 'rush_yds', 'rush_tds']] = df_away_stats['Rush-Yds-TDs'].str.split('-', expand=True)
#         df_away_stats[['cmps','cmp_atts', 'pass_yds', 'pass_tds','ints']] = df_away_stats['Cmp-Att-Yd-TD-INT'].str.split('-', expand=True)
#         df_away_stats[['sacked', 'sacked_yds']] = df_away_stats['Sacked-Yards'].str.split('-', expand=True)
#         df_away_stats[['fumbles', 'fumbles_lost']] = df_away_stats['Fumbles-Lost'].str.split('-', expand=True)
#         df_away_stats[['penalties', 'penalty_yds']] = df_away_stats['Penalties-Yards'].str.split('-', expand=True)
#         df_away_stats[['third_dwn_convert', 'third_downs']] = df_away_stats['Third Down Conv.'].str.split('-', expand=True)
#         df_away_stats[['fourth_dwn_convert', 'fourth_down_atts']] = df_away_stats['Fourth Down Conv.'].str.split('-', expand=True)

#         # Selecting final columns for home team
#         df_hm_stats = df_hm_stats[['Team','Date',"first_downs", 'rush_atts','rush_yds',
#             'rush_tds', 'cmps', 'cmp_atts', 'pass_yds', 'pass_tds', 'ints', 'sacked', 'sacked_yds','net_pass_yds',
#             'total_yds', 'fumbles', 'fumbles_lost','turnovers', 'penalties','penalty_yds', 'third_dwn_convert', 
#             'third_downs', 'fourth_dwn_convert', 'fourth_down_atts', 'time_possession',]]

#         # Selecting final columns for away team
#         df_away_stats = df_away_stats[['Team','Date',"first_downs", 'rush_atts','rush_yds',
#             'rush_tds', 'cmps', 'cmp_atts', 'pass_yds', 'pass_tds', 'ints', 'sacked', 'sacked_yds','net_pass_yds',
#             'total_yds', 'fumbles', 'fumbles_lost','turnovers', 'penalties','penalty_yds', 'third_dwn_convert', 
#             'third_downs', 'fourth_dwn_convert', 'fourth_down_atts', 'time_possession',]]

#         # Convert home team date to string temporarily:
#         df_hm_stats['Date'] = df_hm_stats['Date'].astype(str)

#         # Convert away team date to string temporarily:
#         df_away_stats['Date'] = df_away_stats['Date'].astype(str)

#         # Convert home team dataframe to array:
#         array_hm_stats = df_hm_stats.to_numpy()[0]

#         # Convert away team dataframe to array:
#         array_away_stats = df_away_stats.to_numpy()[0]

#         array_hm_string = ', '.join(map(lambda x: "'" + x + "'", array_hm_stats))
#         array_away_string = ', '.join(map(lambda x: "'" + x + "'", array_away_stats))

#         home_team_insert_statement = f"""
#         INSERT INTO team_weekly_stats (team_name, gametime_date, first_downs, rush_attempts, rush_yards,
#             rush_tds, completions, completion_attempts, pass_yards, pass_tds, interceptions, sacked, sacked_yards, 
#             net_pass_yards, total_yards, fumbles, fumbles_lost, turn_overs, penalties, penalty_yards, third_downs_conversions,
#             third_downs, fourth_down_conversions, fourth_down_attempts, time_of_possession)
#         VALUES (""" + array_hm_string + """);"""

#         away_team_insert_statement = f"""
#         INSERT INTO team_weekly_stats (team_name, gametime_date, first_downs, rush_attempts, rush_yards,
#             rush_tds, completions, completion_attempts, pass_yards, pass_tds, interceptions, sacked, sacked_yards, 
#             net_pass_yards, total_yards, fumbles, fumbles_lost, turn_overs, penalties, penalty_yards, third_downs_conversions,
#             third_downs, fourth_down_conversions, fourth_down_attempts, time_of_possession)
#         VALUES (""" + array_away_string + """);"""

#         with connection, connection.cursor() as cursor:
#             cursor.execute(home_team_insert_statement, row)  
#             cursor.execute(away_team_insert_statement, row)  

#         print("Success")
            
#     except:
#         print("Failure: " + link)
#         filename = "error_log.txt"
#         with open(filename, 'a') as f: 
#             f.write("Error: " + link + ' - ' + str(response) + '\n')

In [29]:
# select_statement = f"""
# SELECT *
# FROM team_weekly_stats;
# """

# sql_player_yearly_stats_df = pd.read_sql_query(select_statement, con = connection)
# sql_player_yearly_stats_df.head(3)

## Team Weekly Stats Table:

In [30]:
team_weekly_stats = pd.read_csv('/dsa/groups/capstonesp2023/online/Team05/pfr_regular_stats.csv', header = 1)

team_weekly_stats = pd.merge(team_weekly_stats, team_df, left_on = ['team'], right_on = ['team_abbr'], how = 'inner')
team_weekly_stats = pd.merge(team_weekly_stats, team_df, left_on = ['opponent'], right_on = ['team_abbr'], how = 'inner')

team_weekly_stats = pd.merge(team_weekly_stats, line_games_df, left_on = ['gametime_day','team_id_x'], right_on = ['gameday','home_team_id'], how = 'left')
team_weekly_stats = pd.merge(team_weekly_stats, line_games_df, left_on = ['gametime_day','team_id_x'], right_on = ['gameday','away_team_id'], how = 'left')

team_weekly_stats['game_id'] = team_weekly_stats[['game_id_x', 'game_id_y']].bfill(axis=1).iloc[:, 0]

team_weekly_stats = pd.merge(team_weekly_stats, sql_game_df, left_on = ['game_id'], right_on = ['game_id'], how = 'inner')

team_weekly_stats['time_of_possession'] = team_weekly_stats['time_of_possession'].str[0:2]

team_weekly_stats = team_weekly_stats[[
       'game_id', 'team_id_x', 'team_name_x', 'team_score', 'team_games_played', 'gametime_year', 'gametime_week', 
       'gametime_day_x', 'team_id_y', 'team_name_y', 'opponent_score', 'home_away', 'win_loss', 'overtime',
       'passing_completions', 'passing_attempts', 'passings_incompletions', 'passing_completion_percentage', 
       'passing_yards', 'passing_touchdowns', 'passing_interceptions', 'passing_touchdown_percentage', 
       'passing_interception_percentage', 'passing_qbr', 'passing_sack', 'passing_sack_yards',
       'passing_sack_percentage', 'passing_yards_per_attempt', 'passing_net_yards_per_attempt', 
       'passing_adjusted_yards_per_attempt', 'passing_adjusted_net_yards_per_attempt',
       'passing_yards_per_completion', 'first_downs_earned', 'rushing_first_downs_earned', 
       'passing_first_downs_earned', 'penalty_first_downs_earned', 'third_down_attempts',
       'third_down_conversions', 'third_down_conversions_percentage', 'fourth_down_attempts',
       'fourth_down_conversions', 'fourth_down_conversions_percentage', 'punts', 'punts_yards',
       'punts_yards_per_punt', 'punts_blocked', 'penalties_committed', 'penalties_committed_yards', 
       'opponent_penalties_committed', 'opponent_penalties_committed_yards', 'rushing_attempts',
       'rushing_yards', 'rushing_yards_per_attempt', 'rushing_touchdowns', 'kickoff_return_touchdowns', 
       'punt_return_touchdowns', 'interception_return_touchdowns', 'fumble_return_touchdowns',
       'other_return_touchdowns', ' total_return_touchdowns', 'total_yardage', 'offensive_plays', 
       'offensive_yards_per_play', 'defensive_plays', 'defensive_yards_per_play', 'turnovers', 
       'touchdowns_scored', 'extra_points_attempted', 'extra_points_made', 'field_goals_attempted', 
       'field_goals_made', 'two_point_conversions_attempted', 'two_point_conversions_made', 'safteys_made',
       'point_differential', 'home_point_differential', 'away_point_differential'
]]


In [31]:
create_statement = f"""
DROP TABLE IF EXISTS team_weekly_stats;

CREATE TABLE team_weekly_stats (
    game_id                                 VARCHAR(50) NOT NULL,
    team_id                                 INT,
    team_name                               VARCHAR(200),
    team_score                              INT,
    team_games_played                       INT,
    gametime_year                           INT,
    gametime_week                           INT,
    gametime_day                            DATE,
    opponent_id                             INT,
    opponent_name                           VARCHAR(200),
    opponent_score                          INT,
    home_or_away                            VARCHAR(50),
    win_or_loss                             VARCHAR(10),
    overtime                                VARCHAR(50),
    passing_completions                     INT,
    passing_attempts                        INT,
    passings_incompletions                  INT,
    passing_completion_percentage           DECIMAL,
    passing_yards                           INT,
    passing_touchdowns                      INT,
    passing_interceptions                   INT,
    passing_touchdown_percentage            DECIMAL,
    passing_interception_percentage         DECIMAL,
    passing_qbr                             DECIMAL,
    passing_sack                            INT,
    passing_sack_yards                      INT,
    passing_sack_percentage                 DECIMAL,
    passing_yards_per_attempt               DECIMAL,
    passing_net_yards_per_attempt           DECIMAL,
    passing_adjusted_yards_per_attempt      DECIMAL,
    passing_adjusted_net_yards_per_attempt  DECIMAL,
    passing_yards_per_completion            DECIMAL,
    first_downs_earned                      INT,
    rushing_first_downs_earned              INT,
    passing_first_downs_earned              INT,
    penalty_first_downs_earned              INT,
    third_down_attempts                     INT,
    third_down_conversions                  INT,
    third_down_conversions_percentage       DECIMAL,
    fourth_down_attempts                    INT,
    fourth_down_conversions                 INT,
    fourth_down_conversions_percentage      DECIMAL,
    punts                                   INT,
    punts_yards                             INT,
    punts_yards_per_punt                    DECIMAL,
    punts_blocked                           INT,
    penalties_committed                     INT,
    penalties_committed_yards               INT,
    opponent_penalties_committed            INT,
    opponent_penalties_committed_yards      INT,
    rushing_attempts                        INT,
    rushing_yards                           INT,
    rushing_yards_per_attempt               DECIMAL,
    rushing_touchdowns                      INT,
    kickoff_return_touchdowns               INT,
    punt_return_touchdowns                  INT,
    interception_return_touchdowns          INT,
    fumble_return_touchdowns                INT,
    other_return_touchdowns                 INT,
    total_return_touchdowns                 INT,
    total_yardage                           INT,
    offensive_plays                         INT,
    offensive_yards_per_play                DECIMAL,
    defensive_plays                         INT,
    defensive_yards_per_play                DECIMAL,
    turnovers                               INT,
    time_of_possession                      INT,
    game_length                             VARCHAR(50),
    touchdowns_scored                       INT,
    extra_points_attempted                  INT,
    extra_points_made                       INT,
    field_goals_attempted                   INT,
    field_goals_made                        INT,
    two_point_conversions_attempted         INT,
    two_point_conversions_made              INT,
    safteys_made                            INT,
    point_differential                      INT,
    home_point_differential                 INT,
    away_point_differential                 INT
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)


In [32]:
insert_statement = f"""
INSERT INTO team_weekly_stats (game_id,team_id,team_name,team_score,team_games_played,gametime_year,gametime_week,gametime_day,
opponent_id,opponent_name,opponent_score,home_or_away,win_or_loss,overtime,passing_completions,passing_attempts,
passings_incompletions,passing_completion_percentage,passing_yards,passing_touchdowns,passing_interceptions,
passing_touchdown_percentage,passing_interception_percentage,passing_qbr,passing_sack,passing_sack_yards,
passing_sack_percentage,passing_yards_per_attempt,passing_net_yards_per_attempt,passing_adjusted_yards_per_attempt,
passing_adjusted_net_yards_per_attempt,passing_yards_per_completion,first_downs_earned,rushing_first_downs_earned,
passing_first_downs_earned,penalty_first_downs_earned,third_down_attempts,third_down_conversions,
third_down_conversions_percentage,fourth_down_attempts,fourth_down_conversions,fourth_down_conversions_percentage,
punts,punts_yards,punts_yards_per_punt,punts_blocked,penalties_committed,penalties_committed_yards,
opponent_penalties_committed,opponent_penalties_committed_yards,rushing_attempts,rushing_yards,
rushing_yards_per_attempt,rushing_touchdowns,kickoff_return_touchdowns,punt_return_touchdowns,
interception_return_touchdowns,fumble_return_touchdowns,other_return_touchdowns,total_return_touchdowns,
total_yardage,offensive_plays,offensive_yards_per_play,defensive_plays,defensive_yards_per_play,turnovers,
touchdowns_scored,extra_points_attempted,extra_points_made,field_goals_attempted,
field_goals_made,two_point_conversions_attempted,two_point_conversions_made,safteys_made,
point_differential, home_point_differential, away_point_differential)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in team_weekly_stats.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)
        

In [33]:
select_statement = f"""
SELECT *
FROM team_weekly_stats;
"""

sql_team_weekly_stats_df = pd.read_sql_query(select_statement, con = connection)
sql_team_weekly_stats_df.head(3)

,game_id,team_id,team_name,team_score,team_games_played,gametime_year,gametime_week,gametime_day,opponent_id,opponent_name,...,extra_points_attempted,extra_points_made,field_goals_attempted,field_goals_made,two_point_conversions_attempted,two_point_conversions_made,safteys_made,point_differential,home_point_differential,away_point_differential
0,2015_01_PIT_NE,3200,New England Patriots,28,1,2015,1,2015-09-10,3900,Pittsburgh Steelers,...,4,4,0,0,0,0,0,7,7,-7
1,2015_01_PIT_NE,3900,Pittsburgh Steelers,21,1,2015,1,2015-09-10,3200,New England Patriots,...,1,1,4,2,1,1,0,7,7,-7
2,2016_07_NE_PIT,3200,New England Patriots,27,7,2016,7,2016-10-23,3900,Pittsburgh Steelers,...,4,3,0,0,0,0,0,11,-11,11


## Team Weekly Running Stats Table:

In [34]:
select_statement = f"""
SELECT game_id, team_id, team_name, team_score, team_games_played, gametime_year, gametime_week, gametime_day, 
    opponent_id, opponent_name, opponent_score, home_or_away, win_or_loss, overtime,
    (SELECT AVG(passing_completions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_completions,
    (SELECT AVG(passing_attempts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_attempts,
    (SELECT AVG(passings_incompletions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passings_incompletions,
    (SELECT AVG(passing_completion_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_completion_percentage,
    (SELECT AVG(passing_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_yards,
    (SELECT AVG(passing_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_touchdowns,
    (SELECT AVG(passing_interceptions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_interceptions,
    (SELECT AVG(passing_touchdown_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_touchdown_percentage,
    (SELECT AVG(passing_interception_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_interception_percentage,
    (SELECT AVG(passing_qbr) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_qbr,
    (SELECT AVG(passing_sack) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_sack,
    (SELECT AVG(passing_sack_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_sack_yards,
    (SELECT AVG(passing_sack_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_sack_percentage,
    (SELECT AVG(passing_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_yards_per_attempt,
    (SELECT AVG(passing_net_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_net_yards_per_attempt,
    (SELECT AVG(passing_adjusted_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_adjusted_yards_per_attempt,
    (SELECT AVG(passing_adjusted_net_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_adjusted_net_yards_per_attempt,
    (SELECT AVG(passing_yards_per_completion) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_yards_per_completion,
    (SELECT AVG(first_downs_earned) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS first_downs_earned,
    (SELECT AVG(rushing_first_downs_earned) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS rushing_first_downs_earned,
    (SELECT AVG(passing_first_downs_earned) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS passing_first_downs_earned,
    (SELECT AVG(penalty_first_downs_earned) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS penalty_first_downs_earned,
    (SELECT AVG(third_down_attempts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS third_down_attempts,
    (SELECT AVG(third_down_conversions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS third_down_conversions,
    (SELECT AVG(third_down_conversions_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS third_down_conversions_percentage,
    (SELECT AVG(fourth_down_attempts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS fourth_down_attempts,
    (SELECT AVG(fourth_down_conversions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS fourth_down_conversions,
    (SELECT AVG(fourth_down_conversions_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS fourth_down_conversions_percentage,
    (SELECT AVG(punts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS punts,
    (SELECT AVG(punts_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS punts_yards,
    (SELECT AVG(punts_yards_per_punt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS punts_yards_per_punt,
    (SELECT AVG(punts_blocked) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS punts_blocked,
    (SELECT AVG(penalties_committed) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS penalties_committed,
    (SELECT AVG(penalties_committed_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS penalties_committed_yards,
    (SELECT AVG(opponent_penalties_committed) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS opponent_penalties_committed,
    (SELECT AVG(opponent_penalties_committed_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS opponent_penalties_committed_yards,
    (SELECT AVG(rushing_attempts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS rushing_attempts,
    (SELECT AVG(rushing_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS rushing_yards,
    (SELECT AVG(rushing_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS rushing_yards_per_attempt,
    (SELECT AVG(rushing_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS rushing_touchdowns,
    (SELECT AVG(kickoff_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS kickoff_return_touchdowns,
    (SELECT AVG(punt_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS punt_return_touchdowns,
    (SELECT AVG(interception_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS interception_return_touchdowns,
    (SELECT AVG(fumble_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS fumble_return_touchdowns,
    (SELECT AVG(other_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS other_return_touchdowns,
    (SELECT AVG(total_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS total_return_touchdowns,
    (SELECT AVG(total_yardage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS total_yardage,
    (SELECT AVG(offensive_plays) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS offensive_plays,
    (SELECT AVG(offensive_yards_per_play) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS offensive_yards_per_play,
    (SELECT AVG(defensive_plays) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS defensive_plays,
    (SELECT AVG(defensive_yards_per_play) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS defensive_yards_per_play,
    (SELECT AVG(turnovers) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS turnovers,
    (SELECT AVG(touchdowns_scored) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS touchdowns_scored,
    (SELECT AVG(extra_points_attempted) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS extra_points_attempted,
    (SELECT AVG(extra_points_made) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS extra_points_made,
    (SELECT AVG(field_goals_attempted) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS field_goals_attempted,
    (SELECT AVG(field_goals_made) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS field_goals_made,
    (SELECT AVG(two_point_conversions_attempted) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS two_point_conversions_attempted,
    (SELECT AVG(two_point_conversions_made) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS two_point_conversions_made,
    (SELECT AVG(safteys_made) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS safteys_made,
    (SELECT SUM(CASE WHEN win_or_loss = 'W' THEN point_differential ELSE point_differential * -1 END) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS point_differential,
    (SELECT SUM(CASE WHEN win_or_loss = 'W' THEN 1 ELSE 0 END) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS team_wins,
    (SELECT SUM(CASE WHEN win_or_loss = 'L' THEN 1 ELSE 0 END) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS team_losses,
    (SELECT SUM(CASE WHEN win_or_loss = 'T' THEN 1 ELSE 0 END) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year AND tws.gametime_day > s.gametime_day) AS team_ties
FROM team_weekly_stats tws
WHERE tws.gametime_week != 1

UNION 

SELECT game_id, team_id, team_name, team_score, team_games_played, gametime_year, gametime_week, gametime_day, 
    opponent_id, opponent_name, opponent_score, home_or_away, win_or_loss, overtime,
    (SELECT AVG(passing_completions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_completions,
    (SELECT AVG(passing_attempts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_attempts,
    (SELECT AVG(passings_incompletions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passings_incompletions,
    (SELECT AVG(passing_completion_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_completion_percentage,
    (SELECT AVG(passing_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_yards,
    (SELECT AVG(passing_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_touchdowns,
    (SELECT AVG(passing_interceptions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_interceptions,
    (SELECT AVG(passing_touchdown_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_touchdown_percentage,
    (SELECT AVG(passing_interception_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_interception_percentage,
    (SELECT AVG(passing_qbr) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_qbr,
    (SELECT AVG(passing_sack) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_sack,
    (SELECT AVG(passing_sack_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_sack_yards,
    (SELECT AVG(passing_sack_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_sack_percentage,
    (SELECT AVG(passing_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_yards_per_attempt,
    (SELECT AVG(passing_net_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_net_yards_per_attempt,
    (SELECT AVG(passing_adjusted_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_adjusted_yards_per_attempt,
    (SELECT AVG(passing_adjusted_net_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_adjusted_net_yards_per_attempt,
    (SELECT AVG(passing_yards_per_completion) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_yards_per_completion,
    (SELECT AVG(first_downs_earned) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS first_downs_earned,
    (SELECT AVG(rushing_first_downs_earned) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS rushing_first_downs_earned,
    (SELECT AVG(passing_first_downs_earned) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS passing_first_downs_earned,
    (SELECT AVG(penalty_first_downs_earned) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS penalty_first_downs_earned,
    (SELECT AVG(third_down_attempts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS third_down_attempts,
    (SELECT AVG(third_down_conversions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS third_down_conversions,
    (SELECT AVG(third_down_conversions_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS third_down_conversions_percentage,
    (SELECT AVG(fourth_down_attempts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS fourth_down_attempts,
    (SELECT AVG(fourth_down_conversions) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS fourth_down_conversions,
    (SELECT AVG(fourth_down_conversions_percentage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS fourth_down_conversions_percentage,
    (SELECT AVG(punts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS punts,
    (SELECT AVG(punts_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS punts_yards,
    (SELECT AVG(punts_yards_per_punt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS punts_yards_per_punt,
    (SELECT AVG(punts_blocked) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS punts_blocked,
    (SELECT AVG(penalties_committed) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS penalties_committed,
    (SELECT AVG(penalties_committed_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS penalties_committed_yards,
    (SELECT AVG(opponent_penalties_committed) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS opponent_penalties_committed,
    (SELECT AVG(opponent_penalties_committed_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS opponent_penalties_committed_yards,
    (SELECT AVG(rushing_attempts) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS rushing_attempts,
    (SELECT AVG(rushing_yards) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS rushing_yards,
    (SELECT AVG(rushing_yards_per_attempt) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS rushing_yards_per_attempt,
    (SELECT AVG(rushing_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS rushing_touchdowns,
    (SELECT AVG(kickoff_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS kickoff_return_touchdowns,
    (SELECT AVG(punt_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS punt_return_touchdowns,
    (SELECT AVG(interception_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS interception_return_touchdowns,
    (SELECT AVG(fumble_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS fumble_return_touchdowns,
    (SELECT AVG(other_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS other_return_touchdowns,
    (SELECT AVG(total_return_touchdowns) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS total_return_touchdowns,
    (SELECT AVG(total_yardage) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS total_yardage,
    (SELECT AVG(offensive_plays) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS offensive_plays,
    (SELECT AVG(offensive_yards_per_play) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS offensive_yards_per_play,
    (SELECT AVG(defensive_plays) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS defensive_plays,
    (SELECT AVG(defensive_yards_per_play) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS defensive_yards_per_play,
    (SELECT AVG(turnovers) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS turnovers,
    (SELECT AVG(touchdowns_scored) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS touchdowns_scored,
    (SELECT AVG(extra_points_attempted) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS extra_points_attempted,
    (SELECT AVG(extra_points_made) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS extra_points_made,
    (SELECT AVG(field_goals_attempted) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS field_goals_attempted,
    (SELECT AVG(field_goals_made) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS field_goals_made,
    (SELECT AVG(two_point_conversions_attempted) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS two_point_conversions_attempted,
    (SELECT AVG(two_point_conversions_made) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS two_point_conversions_made,
    (SELECT AVG(safteys_made) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS safteys_made,
    (SELECT SUM(CASE WHEN win_or_loss = 'W' THEN point_differential ELSE point_differential * -1 END) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS point_differential,
    (SELECT SUM(CASE WHEN win_or_loss = 'W' THEN 1 ELSE 0 END) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS team_wins,
    (SELECT SUM(CASE WHEN win_or_loss = 'L' THEN 1 ELSE 0 END) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS team_losses,
    (SELECT SUM(CASE WHEN win_or_loss = 'T' THEN 1 ELSE 0 END) FROM team_weekly_stats s WHERE tws.team_id = s.team_id AND tws.gametime_year = s.gametime_year - 1) AS team_ties
FROM team_weekly_stats tws
WHERE tws.gametime_week = 1
"""

team_weekly_stats_running_df = pd.read_sql_query(select_statement, con = connection)

In [35]:
create_statement = f"""
DROP TABLE IF EXISTS team_weekly_stats_running;

CREATE TABLE team_weekly_stats_running (
    game_id                                 VARCHAR(50) NOT NULL,
    team_id                                 INT,
    team_name                               VARCHAR(200),
    team_score                              INT,
    team_games_played                       INT,
    gametime_year                           INT,
    gametime_week                           INT,
    gametime_day                            DATE,
    opponent_id                             INT,
    opponent_name                           VARCHAR(200),
    opponent_score                          INT,
    home_or_away                            VARCHAR(50),
    win_or_loss                             VARCHAR(10),
    overtime                                VARCHAR(50),
    passing_completions                     DECIMAL,
    passing_attempts                        DECIMAL,
    passings_incompletions                  DECIMAL,
    passing_completion_percentage           DECIMAL,
    passing_yards                           DECIMAL,
    passing_touchdowns                      DECIMAL,
    passing_interceptions                   DECIMAL,
    passing_touchdown_percentage            DECIMAL,
    passing_interception_percentage         DECIMAL,
    passing_qbr                             DECIMAL,
    passing_sack                            DECIMAL,
    passing_sack_yards                      DECIMAL,
    passing_sack_percentage                 DECIMAL,
    passing_yards_per_attempt               DECIMAL,
    passing_net_yards_per_attempt           DECIMAL,
    passing_adjusted_yards_per_attempt      DECIMAL,
    passing_adjusted_net_yards_per_attempt  DECIMAL,
    passing_yards_per_completion            DECIMAL,
    first_downs_earned                      DECIMAL,
    rushing_first_downs_earned              DECIMAL,
    passing_first_downs_earned              DECIMAL,
    penalty_first_downs_earned              DECIMAL,
    third_down_attempts                     DECIMAL,
    third_down_conversions                  DECIMAL,
    third_down_conversions_percentage       DECIMAL,
    fourth_down_attempts                    DECIMAL,
    fourth_down_conversions                 DECIMAL,
    fourth_down_conversions_percentage      DECIMAL,
    punts                                   DECIMAL,
    punts_yards                             DECIMAL,
    punts_yards_per_punt                    DECIMAL,
    punts_blocked                           DECIMAL,
    penalties_committed                     DECIMAL,
    penalties_committed_yards               DECIMAL,
    opponent_penalties_committed            DECIMAL,
    opponent_penalties_committed_yards      DECIMAL,
    rushing_attempts                        DECIMAL,
    rushing_yards                           DECIMAL,
    rushing_yards_per_attempt               DECIMAL,
    rushing_touchdowns                      DECIMAL,
    kickoff_return_touchdowns               DECIMAL,
    punt_return_touchdowns                  DECIMAL,
    interception_return_touchdowns          DECIMAL,
    fumble_return_touchdowns                DECIMAL,
    other_return_touchdowns                 DECIMAL,
    total_return_touchdowns                 DECIMAL,
    total_yardage                           DECIMAL,
    offensive_plays                         DECIMAL,
    offensive_yards_per_play                DECIMAL,
    defensive_plays                         DECIMAL,
    defensive_yards_per_play                DECIMAL,
    turnovers                               DECIMAL,
    touchdowns_scored                       DECIMAL,
    extra_points_attempted                  DECIMAL,
    extra_points_made                       DECIMAL,
    field_goals_attempted                   DECIMAL,
    field_goals_made                        DECIMAL,
    two_point_conversions_attempted         DECIMAL,
    two_point_conversions_made              DECIMAL,
    safteys_made                            DECIMAL,
    point_differential                      DECIMAL,
    team_wins                               DECIMAL,
    team_losses                             DECIMAL,
    team_ties                               DECIMAL
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [36]:
insert_statement = f"""
INSERT INTO team_weekly_stats_running (game_id,team_id,team_name,team_score,team_games_played,gametime_year,gametime_week,gametime_day,
opponent_id,opponent_name,opponent_score,home_or_away,win_or_loss,overtime,passing_completions,passing_attempts,
passings_incompletions,passing_completion_percentage,passing_yards,passing_touchdowns,passing_interceptions,
passing_touchdown_percentage,passing_interception_percentage,passing_qbr,passing_sack,passing_sack_yards,
passing_sack_percentage,passing_yards_per_attempt,passing_net_yards_per_attempt,passing_adjusted_yards_per_attempt,
passing_adjusted_net_yards_per_attempt,passing_yards_per_completion,first_downs_earned,rushing_first_downs_earned,
passing_first_downs_earned,penalty_first_downs_earned,third_down_attempts,third_down_conversions,
third_down_conversions_percentage,fourth_down_attempts,fourth_down_conversions,fourth_down_conversions_percentage,
punts,punts_yards,punts_yards_per_punt,punts_blocked,penalties_committed,penalties_committed_yards,
opponent_penalties_committed,opponent_penalties_committed_yards,rushing_attempts,rushing_yards,
rushing_yards_per_attempt,rushing_touchdowns,kickoff_return_touchdowns,punt_return_touchdowns,
interception_return_touchdowns,fumble_return_touchdowns,other_return_touchdowns,total_return_touchdowns,
total_yardage,offensive_plays,offensive_yards_per_play,defensive_plays,defensive_yards_per_play,turnovers,
touchdowns_scored,extra_points_attempted,extra_points_made,field_goals_attempted,
field_goals_made,two_point_conversions_attempted,two_point_conversions_made,safteys_made,point_differential,
team_wins,team_losses,team_ties)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in team_weekly_stats_running_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [37]:
select_statement = f"""
SELECT *
FROM team_weekly_stats_running;
"""

sql_team_weekly_stats_df = pd.read_sql_query(select_statement, con = connection)
sql_team_weekly_stats_df.head(3)

,game_id,team_id,team_name,team_score,team_games_played,gametime_year,gametime_week,gametime_day,opponent_id,opponent_name,...,extra_points_made,field_goals_attempted,field_goals_made,two_point_conversions_attempted,two_point_conversions_made,safteys_made,point_differential,team_wins,team_losses,team_ties
0,2016_13_MIA_BAL,2700,Miami Dolphins,6,12,2016,13,2016-12-04,325,Baltimore Ravens,...,2.545455,1.454545,1.181818,0.090909,0.090909,0.0,9.0,7.0,4.0,0.0
1,2021_04_TB_NE,4900,Tampa Bay Buccaneers,19,4,2021,4,2021-10-03,3200,New England Patriots,...,4.333333,1.000000,0.666667,0.000000,0.000000,0.0,15.0,2.0,1.0,0.0
2,2021_04_NYG_NO,3410,New York Giants,27,4,2021,4,2021-10-03,3300,New Orleans Saints,...,1.000000,2.333333,2.333333,0.333333,0.333333,0.0,-18.0,0.0,3.0,0.0


## Team Weekly Advanced Stats Table:

In [38]:
team_weekly_advanced_stats = pd.read_csv('/dsa/groups/capstonesp2023/online/Team05/pfr_advanced_stats.csv', header = 0)

team_weekly_advanced_stats = pd.merge(team_weekly_advanced_stats, team_df, left_on = ['team'], right_on = ['team_abbr'], how = 'inner')
team_weekly_advanced_stats = pd.merge(team_weekly_advanced_stats, team_df, left_on = ['opponent'], right_on = ['team_abbr'], how = 'inner')

team_weekly_advanced_stats = pd.merge(team_weekly_advanced_stats, line_games_df, left_on = ['gametime_day','team_id_x'], right_on = ['gameday','home_team_id'], how = 'left')
team_weekly_advanced_stats = pd.merge(team_weekly_advanced_stats, line_games_df, left_on = ['gametime_day','team_id_x'], right_on = ['gameday','away_team_id'], how = 'left')

team_weekly_advanced_stats['game_id'] = team_weekly_advanced_stats[['game_id_x', 'game_id_y']].bfill(axis=1).iloc[:, 0]

team_weekly_advanced_stats = pd.merge(team_weekly_advanced_stats, sql_game_df, left_on = ['game_id'], right_on = ['game_id'], how = 'inner')

team_weekly_advanced_stats = team_weekly_advanced_stats[[
    'game_id', 'team_id_x', 'team_name_x', 'team_id_y','team_name_y', 'home_or_away', 'gametime_year',
    'gametime_week', 'gametime_day_x', 'passing_iay', 'passing_iay_over_pa', 'passing_cay', 'passing_cay_over_cmp',
    'passing_cay_over_pa', 'passing_yac', 'passing_yac_over_cmp', 'pass_coverage_dadot', 'pass_coverage_air',
    'pass_coverage_yac', 'pass_rush_blitz', 'pass_rush_bats', 'pass_rush_hurries', 'pass_rush_qbkd',
    'pass_rush_prss', 'score_by_quarter_q1', 'score_by_quarter_q2', 'score_by_quarter_q3', 'score_by_quarter_q4',
    'opponent_score_by_quarter_q1', 'opponent_score_by_quarter_q2', 'opponent_score_by_quarter_q3',
    'opponent_score_by_quarter_q4', 'rpo_plays', 'rpo_yards', 'rpo_pass_attempts', 
    'rpo_pass_yards', 'rpo_rush_attempts', 'rpo_rush_yards', 'play_action_pass_attempts', 'play_action_pass_yards', 
    'passing_bats', 'passing_throw_aways', 'passing_spikes', 'passing_drops', 'passing_drop_percentage',
    'passing_bad_throw', 'passing_bad_throw_percentage', 'passing_on_target', 'passing_on_target_percentage',
    'rushing_ybc', 'rushing_ybc_over_attempts', 'rushing_yax', 'rushing_yac_over_attempts', 'rushing_brk_tkl',
    'rushing_attempts_over_br', 'receiving_yac', 'receiving_adot', 'receiving_brk_tkl', 'receiving_rec_over_br',
    'receiving_drops', 'receiving_drop_percentage', 'passing_pkt_time', 'passing_blitz', 'passing_hurries', 
    'passing_hits', 'passing_prss', 'passing_prss_percentage', 'passing_scrm', 'passing_yards_over_scr'
]]


In [39]:
create_statement = f"""
DROP TABLE IF EXISTS team_weekly_advanced_stats;

CREATE TABLE team_weekly_advanced_stats (
    game_id                                            VARCHAR(50) NOT NULL,
    team_id                                            INT,
    team_name                                          VARCHAR(200),
    opponent_id                                        INT,
    opponent_name                                      VARCHAR(200),
    home_or_away                                       VARCHAR(50),
    gametime_year                                      INT,
    gametime_week                                      INT,
    gametime_day                                       DATE,
    pass_intended_air_yards                            INT,
    passing_intended_air_yards_per_pass_attempt        DECIMAL,
    passing_completed_air_yards                        INT,
    passing_completed_air_yards_per_pass_completion    DECIMAL,
    passing_completed_air_yards_per_pass_attempt       DECIMAL,
    passing_yards_after_catch                          INT,
    passing_yards_after_catch_per_pass_completion      DECIMAL,
    passing_coverage_avg_depth_of_target               DECIMAL,
    passing_coverage_air_yards_allowed                 INT,
    passing_coverage_yards_after_catch_allowed         INT,
    pass_rush_blitz                                    INT,
    pass_rush_bats                                     INT,
    pass_rush_qb_hurries                               INT,
    pass_rush_qb_knockdowns                            INT,
    pass_rush_pressures                                INT,
    team_quarter_one_points_scored                     INT,
    team_quarter_two_points_scored                     INT,
    team_quarter_three_points_scored                   INT,
    team_quarter_four_points_scored                    INT,
    opponents_quarter_one_points_scored                INT,
    opponents_quarter_two_points_scored                INT,
    opponents_quarter_three_points_scored              INT,
    opponents_quarter_four_points_scored               INT,
    run_pass_option_plays_run                          INT,
    run_pass_option_yards                              INT,
    run_pass_option_pass_attempts                      INT,
    run_pass_option_pass_yards                         INT,
    run_pass_option_rush_attempts                      INT,
    run_pass_option_rush_yards                         INT,
    play_action_pass_attempts                          INT,
    play_action_pass_yards                             INT,
    passing_batted_passes                              INT,
    passing_throw_aways                                INT,
    passing_spikes                                     INT,
    passing_drops                                      INT,
    passing_drop_percentage                            DECIMAL,
    passing_poor_throw                                 INT,
    passing_poor_throw_percentage                      DECIMAL,
    passing_on_target_pass                             INT,
    passing_on_target_pass_percentage                  DECIMAL,
    rushing_yards_before_contact                       INT,
    rushing_yards_before_contact_per_rush_attempt      DECIMAL,
    rushing_yards_after_contact                        INT,
    rushing_yards_after_contact_per_rush_attempt       DECIMAL,
    rushing_broken_tackle                              INT,
    rushing_rushes_per_broken_tackle                   DECIMAL,
    receiving_yards_after_catch                        INT,
    receiving_avg_depth_of_target                      DECIMAL,
    receiving_broken_tackle                            INT,
    receiving_receptions_per_broken_tackle             DECIMAL,
    receiving_drop                                     INT,
    receiving_drop_percentage                          DECIMAL,
    passing_avg_pocket_time                            DECIMAL,
    passing_times_blitzed                              INT,
    passing_times_hurried                              INT,
    passing_times_qb_hit                               INT,
    passing_times_qb_pressured                         INT,
    passing_qb_pressured_percentage                    DECIMAL,
    passing_scrambles                                  INT,
    passing_yards_per_scramble                         DECIMAL
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [40]:
insert_statement = f"""
INSERT INTO team_weekly_advanced_stats (game_id, team_id, team_name, opponent_id, opponent_name, home_or_away, 
gametime_year, gametime_week, gametime_day, pass_intended_air_yards, passing_intended_air_yards_per_pass_attempt, 
passing_completed_air_yards, passing_completed_air_yards_per_pass_completion, 
passing_completed_air_yards_per_pass_attempt, passing_yards_after_catch, 
passing_yards_after_catch_per_pass_completion, passing_coverage_avg_depth_of_target,
passing_coverage_air_yards_allowed, passing_coverage_yards_after_catch_allowed, pass_rush_blitz, pass_rush_bats,
pass_rush_qb_hurries, pass_rush_qb_knockdowns, pass_rush_pressures, team_quarter_one_points_scored,
team_quarter_two_points_scored, team_quarter_three_points_scored, team_quarter_four_points_scored,
opponents_quarter_one_points_scored, opponents_quarter_two_points_scored, opponents_quarter_three_points_scored,
opponents_quarter_four_points_scored, run_pass_option_plays_run, run_pass_option_yards,
run_pass_option_pass_attempts, run_pass_option_pass_yards, run_pass_option_rush_attempts, 
run_pass_option_rush_yards, play_action_pass_attempts, play_action_pass_yards, passing_batted_passes,
passing_throw_aways, passing_spikes, passing_drops, passing_drop_percentage, passing_poor_throw,
passing_poor_throw_percentage, passing_on_target_pass, passing_on_target_pass_percentage, 
rushing_yards_before_contact, rushing_yards_before_contact_per_rush_attempt, rushing_yards_after_contact,
rushing_yards_after_contact_per_rush_attempt, rushing_broken_tackle, rushing_rushes_per_broken_tackle,
receiving_yards_after_catch, receiving_avg_depth_of_target, receiving_broken_tackle, 
receiving_receptions_per_broken_tackle, receiving_drop, receiving_drop_percentage, passing_avg_pocket_time,
passing_times_blitzed, passing_times_hurried, passing_times_qb_hit, passing_times_qb_pressured,
passing_qb_pressured_percentage, passing_scrambles, passing_yards_per_scramble)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in team_weekly_advanced_stats.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [41]:
select_statement = f"""
SELECT *
FROM team_weekly_advanced_stats;
"""

sql_team_weekly_advanced_stats_df = pd.read_sql_query(select_statement, con = connection)
sql_team_weekly_advanced_stats_df.head(3)

,game_id,team_id,team_name,opponent_id,opponent_name,home_or_away,gametime_year,gametime_week,gametime_day,pass_intended_air_yards,...,receiving_drop,receiving_drop_percentage,passing_avg_pocket_time,passing_times_blitzed,passing_times_hurried,passing_times_qb_hit,passing_times_qb_pressured,passing_qb_pressured_percentage,passing_scrambles,passing_yards_per_scramble
0,2018_02_CAR_ATL,200,Atlanta Falcons,750,Carolina Panthers,home,2018,2,2018-09-16,293,...,2,7.1,2.2,7,0,2,2,6.5,3,5.7
1,2018_02_CAR_ATL,750,Carolina Panthers,200,Atlanta Falcons,away,2018,2,2018-09-16,324,...,5,11.1,2.4,6,3,2,7,14.0,3,9.3
2,2019_11_ATL_CAR,200,Atlanta Falcons,750,Carolina Panthers,away,2019,11,2019-11-17,292,...,3,9.4,2.5,8,2,4,9,25.7,0,NaN


## Team Weekly Running Advanced Stats Table:

In [42]:
select_statement = f"""
SELECT game_id, team_id, team_name, opponent_id, opponent_name, home_or_away, gametime_year, gametime_week, gametime_day,
    (SELECT AVG(pass_intended_air_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS pass_intended_air_yards,
    (SELECT AVG(passing_intended_air_yards_per_pass_attempt) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_intended_air_yards_per_pass_attempt,
    (SELECT AVG(passing_completed_air_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_completed_air_yards,
    (SELECT AVG(passing_completed_air_yards_per_pass_completion) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_completed_air_yards_per_pass_completion,
    (SELECT AVG(passing_completed_air_yards_per_pass_attempt) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_completed_air_yards_per_pass_attempt,
    (SELECT AVG(passing_yards_after_catch) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_yards_after_catch,
    (SELECT AVG(passing_yards_after_catch_per_pass_completion) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_yards_after_catch_per_pass_completion,
    (SELECT AVG(passing_coverage_avg_depth_of_target) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_coverage_avg_depth_of_target,
    (SELECT AVG(passing_coverage_air_yards_allowed) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_coverage_air_yards_allowed,
    (SELECT AVG(passing_coverage_yards_after_catch_allowed) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_coverage_yards_after_catch_allowed,
    (SELECT AVG(pass_rush_blitz) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS pass_rush_blitz,
    (SELECT AVG(pass_rush_bats) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS pass_rush_bats,
    (SELECT AVG(pass_rush_qb_hurries) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS pass_rush_qb_hurries,
    (SELECT AVG(pass_rush_qb_knockdowns) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS pass_rush_qb_knockdowns,
    (SELECT AVG(pass_rush_pressures) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS pass_rush_pressures,
    (SELECT AVG(team_quarter_one_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS team_quarter_one_points_scored,
    (SELECT AVG(team_quarter_two_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS team_quarter_two_points_scored,
    (SELECT AVG(team_quarter_three_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS team_quarter_three_points_scored,
    (SELECT AVG(team_quarter_four_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS team_quarter_four_points_scored,
    (SELECT AVG(opponents_quarter_one_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS opponents_quarter_one_points_scored,
    (SELECT AVG(opponents_quarter_two_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS opponents_quarter_two_points_scored,
    (SELECT AVG(opponents_quarter_three_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS opponents_quarter_three_points_scored,
    (SELECT AVG(opponents_quarter_four_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS opponents_quarter_four_points_scored,
    (SELECT AVG(run_pass_option_plays_run) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS run_pass_option_plays_run,
    (SELECT AVG(run_pass_option_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS run_pass_option_yards,
    (SELECT AVG(run_pass_option_pass_attempts) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS run_pass_option_pass_attempts,
    (SELECT AVG(run_pass_option_pass_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS run_pass_option_pass_yards,
    (SELECT AVG(run_pass_option_rush_attempts) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS run_pass_option_rush_attempts,
    (SELECT AVG(run_pass_option_rush_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS run_pass_option_rush_yards,
    (SELECT AVG(play_action_pass_attempts) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS play_action_pass_attempts,
    (SELECT AVG(play_action_pass_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS play_action_pass_yards,
    (SELECT AVG(passing_batted_passes) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_batted_passes,
    (SELECT AVG(passing_throw_aways) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_throw_aways,
    (SELECT AVG(passing_spikes) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_spikes,
    (SELECT AVG(passing_drops) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_drops,
    (SELECT AVG(passing_drop_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_drop_percentage,
    (SELECT AVG(passing_poor_throw) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_poor_throw,
    (SELECT AVG(passing_poor_throw_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_poor_throw_percentage,
    (SELECT AVG(passing_on_target_pass) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_on_target_pass,
    (SELECT AVG(passing_on_target_pass_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_on_target_pass_percentage,
    (SELECT AVG(rushing_yards_before_contact) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS rushing_yards_before_contact,
    (SELECT AVG(rushing_yards_before_contact_per_rush_attempt) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS rushing_yards_before_contact_per_rush_attempt,
    (SELECT AVG(rushing_yards_after_contact) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS rushing_yards_after_contact,
    (SELECT AVG(rushing_yards_after_contact_per_rush_attempt) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS rushing_yards_after_contact_per_rush_attempt,
    (SELECT AVG(rushing_broken_tackle) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS rushing_broken_tackle,
    (SELECT AVG(rushing_rushes_per_broken_tackle) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS rushing_rushes_per_broken_tackle,
    (SELECT AVG(receiving_yards_after_catch) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS receiving_yards_after_catch,
    (SELECT AVG(receiving_avg_depth_of_target) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS receiving_avg_depth_of_target,
    (SELECT AVG(receiving_broken_tackle) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS receiving_broken_tackle,
    (SELECT AVG(receiving_receptions_per_broken_tackle) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS receiving_receptions_per_broken_tackle,
    (SELECT AVG(receiving_drop) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS receiving_drop,
    (SELECT AVG(receiving_drop_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS receiving_drop_percentage,
    (SELECT AVG(passing_avg_pocket_time) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_avg_pocket_time,
    (SELECT AVG(passing_times_blitzed) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_times_blitzed,
    (SELECT AVG(passing_times_hurried) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_times_hurried,
    (SELECT AVG(passing_times_qb_hit) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_times_qb_hit,
    (SELECT AVG(passing_times_qb_pressured) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_times_qb_pressured,
    (SELECT AVG(passing_qb_pressured_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_qb_pressured_percentage,
    (SELECT AVG(passing_scrambles) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_scrambles,
    (SELECT AVG(passing_yards_per_scramble) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year AND twas.gametime_day > s.gametime_day) AS passing_yards_per_scramble
FROM team_weekly_advanced_stats twas
WHERE twas.gametime_week != 1

UNION 

SELECT game_id, team_id, team_name, opponent_id, opponent_name, home_or_away, gametime_year, gametime_week, gametime_day,
    (SELECT AVG(pass_intended_air_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS pass_intended_air_yards,
    (SELECT AVG(passing_intended_air_yards_per_pass_attempt) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_intended_air_yards_per_pass_attempt,
    (SELECT AVG(passing_completed_air_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_completed_air_yards,
    (SELECT AVG(passing_completed_air_yards_per_pass_completion) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_completed_air_yards_per_pass_completion,
    (SELECT AVG(passing_completed_air_yards_per_pass_attempt) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_completed_air_yards_per_pass_attempt,
    (SELECT AVG(passing_yards_after_catch) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_yards_after_catch,
    (SELECT AVG(passing_yards_after_catch_per_pass_completion) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_yards_after_catch_per_pass_completion,
    (SELECT AVG(passing_coverage_avg_depth_of_target) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_coverage_avg_depth_of_target,
    (SELECT AVG(passing_coverage_air_yards_allowed) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_coverage_air_yards_allowed,
    (SELECT AVG(passing_coverage_yards_after_catch_allowed) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_coverage_yards_after_catch_allowed,
    (SELECT AVG(pass_rush_blitz) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS pass_rush_blitz,
    (SELECT AVG(pass_rush_bats) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS pass_rush_bats,
    (SELECT AVG(pass_rush_qb_hurries) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS pass_rush_qb_hurries,
    (SELECT AVG(pass_rush_qb_knockdowns) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS pass_rush_qb_knockdowns,
    (SELECT AVG(pass_rush_pressures) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS pass_rush_pressures,
    (SELECT AVG(team_quarter_one_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS team_quarter_one_points_scored,
    (SELECT AVG(team_quarter_two_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS team_quarter_two_points_scored,
    (SELECT AVG(team_quarter_three_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS team_quarter_three_points_scored,
    (SELECT AVG(team_quarter_four_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS team_quarter_four_points_scored,
    (SELECT AVG(opponents_quarter_one_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS opponents_quarter_one_points_scored,
    (SELECT AVG(opponents_quarter_two_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS opponents_quarter_two_points_scored,
    (SELECT AVG(opponents_quarter_three_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS opponents_quarter_three_points_scored,
    (SELECT AVG(opponents_quarter_four_points_scored) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS opponents_quarter_four_points_scored,
    (SELECT AVG(run_pass_option_plays_run) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS run_pass_option_plays_run,
    (SELECT AVG(run_pass_option_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS run_pass_option_yards,
    (SELECT AVG(run_pass_option_pass_attempts) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS run_pass_option_pass_attempts,
    (SELECT AVG(run_pass_option_pass_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS run_pass_option_pass_yards,
    (SELECT AVG(run_pass_option_rush_attempts) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS run_pass_option_rush_attempts,
    (SELECT AVG(run_pass_option_rush_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS run_pass_option_rush_yards,
    (SELECT AVG(play_action_pass_attempts) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS play_action_pass_attempts,
    (SELECT AVG(play_action_pass_yards) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS play_action_pass_yards,
    (SELECT AVG(passing_batted_passes) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_batted_passes,
    (SELECT AVG(passing_throw_aways) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_throw_aways,
    (SELECT AVG(passing_spikes) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_spikes,
    (SELECT AVG(passing_drops) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_drops,
    (SELECT AVG(passing_drop_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_drop_percentage,
    (SELECT AVG(passing_poor_throw) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_poor_throw,
    (SELECT AVG(passing_poor_throw_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_poor_throw_percentage,
    (SELECT AVG(passing_on_target_pass) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_on_target_pass,
    (SELECT AVG(passing_on_target_pass_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_on_target_pass_percentage,
    (SELECT AVG(rushing_yards_before_contact) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS rushing_yards_before_contact,
    (SELECT AVG(rushing_yards_before_contact_per_rush_attempt) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS rushing_yards_before_contact_per_rush_attempt,
    (SELECT AVG(rushing_yards_after_contact) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS rushing_yards_after_contact,
    (SELECT AVG(rushing_yards_after_contact_per_rush_attempt) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS rushing_yards_after_contact_per_rush_attempt,
    (SELECT AVG(rushing_broken_tackle) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS rushing_broken_tackle,
    (SELECT AVG(rushing_rushes_per_broken_tackle) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS rushing_rushes_per_broken_tackle,
    (SELECT AVG(receiving_yards_after_catch) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS receiving_yards_after_catch,
    (SELECT AVG(receiving_avg_depth_of_target) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS receiving_avg_depth_of_target,
    (SELECT AVG(receiving_broken_tackle) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS receiving_broken_tackle,
    (SELECT AVG(receiving_receptions_per_broken_tackle) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS receiving_receptions_per_broken_tackle,
    (SELECT AVG(receiving_drop) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS receiving_drop,
    (SELECT AVG(receiving_drop_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS receiving_drop_percentage,
    (SELECT AVG(passing_avg_pocket_time) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_avg_pocket_time,
    (SELECT AVG(passing_times_blitzed) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_times_blitzed,
    (SELECT AVG(passing_times_hurried) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_times_hurried,
    (SELECT AVG(passing_times_qb_hit) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_times_qb_hit,
    (SELECT AVG(passing_times_qb_pressured) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_times_qb_pressured,
    (SELECT AVG(passing_qb_pressured_percentage) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_qb_pressured_percentage,
    (SELECT AVG(passing_scrambles) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_scrambles,
    (SELECT AVG(passing_yards_per_scramble) FROM team_weekly_advanced_stats s WHERE twas.team_id = s.team_id AND twas.gametime_year = s.gametime_year - 1) AS passing_yards_per_scramble
FROM team_weekly_advanced_stats twas
WHERE twas.gametime_week = 1
"""

team_weekly_advanced_stats_running_df = pd.read_sql_query(select_statement, con = connection)

In [44]:
create_statement = f"""
DROP TABLE IF EXISTS team_weekly_advanced_stats_running;

CREATE TABLE team_weekly_advanced_stats_running (
    game_id                                            VARCHAR(50) NOT NULL,
    team_id                                            INT,
    team_name                                          VARCHAR(200),
    opponent_id                                        INT,
    opponent_name                                      VARCHAR(200),
    home_or_away                                       VARCHAR(50),
    gametime_year                                      INT,
    gametime_week                                      INT,
    gametime_day                                       DATE,
    pass_intended_air_yards                            DECIMAL,
    passing_intended_air_yards_per_pass_attempt        DECIMAL,
    passing_completed_air_yards                        DECIMAL,
    passing_completed_air_yards_per_pass_completion    DECIMAL,
    passing_completed_air_yards_per_pass_attempt       DECIMAL,
    passing_yards_after_catch                          DECIMAL,
    passing_yards_after_catch_per_pass_completion      DECIMAL,
    passing_coverage_avg_depth_of_target               DECIMAL,
    passing_coverage_air_yards_allowed                 DECIMAL,
    passing_coverage_yards_after_catch_allowed         DECIMAL,
    pass_rush_blitz                                    DECIMAL,
    pass_rush_bats                                     DECIMAL,
    pass_rush_qb_hurries                               DECIMAL,
    pass_rush_qb_knockdowns                            DECIMAL,
    pass_rush_pressures                                DECIMAL,
    team_quarter_one_points_scored                     DECIMAL,
    team_quarter_two_points_scored                     DECIMAL,
    team_quarter_three_points_scored                   DECIMAL,
    team_quarter_four_points_scored                    DECIMAL,
    opponents_quarter_one_points_scored                DECIMAL,
    opponents_quarter_two_points_scored                DECIMAL,
    opponents_quarter_three_points_scored              DECIMAL,
    opponents_quarter_four_points_scored               DECIMAL,
    run_pass_option_plays_run                          DECIMAL,
    run_pass_option_yards                              DECIMAL,
    run_pass_option_pass_attempts                      DECIMAL,
    run_pass_option_pass_yards                         DECIMAL,
    run_pass_option_rush_attempts                      DECIMAL,
    run_pass_option_rush_yards                         DECIMAL,
    play_action_pass_attempts                          DECIMAL,
    play_action_pass_yards                             DECIMAL,
    passing_batted_passes                              DECIMAL,
    passing_throw_aways                                DECIMAL,
    passing_spikes                                     DECIMAL,
    passing_drops                                      DECIMAL,
    passing_drop_percentage                            DECIMAL,
    passing_poor_throw                                 DECIMAL,
    passing_poor_throw_percentage                      DECIMAL,
    passing_on_target_pass                             DECIMAL,
    passing_on_target_pass_percentage                  DECIMAL,
    rushing_yards_before_contact                       DECIMAL,
    rushing_yards_before_contact_per_rush_attempt      DECIMAL,
    rushing_yards_after_contact                        DECIMAL,
    rushing_yards_after_contact_per_rush_attempt       DECIMAL,
    rushing_broken_tackle                              DECIMAL,
    rushing_rushes_per_broken_tackle                   DECIMAL,
    receiving_yards_after_catch                        DECIMAL,
    receiving_avg_depth_of_target                      DECIMAL,
    receiving_broken_tackle                            DECIMAL,
    receiving_receptions_per_broken_tackle             DECIMAL,
    receiving_drop                                     DECIMAL,
    receiving_drop_percentage                          DECIMAL,
    passing_avg_pocket_time                            DECIMAL,
    passing_times_blitzed                              DECIMAL,
    passing_times_hurried                              DECIMAL,
    passing_times_qb_hit                               DECIMAL,
    passing_times_qb_pressured                         DECIMAL,
    passing_qb_pressured_percentage                    DECIMAL,
    passing_scrambles                                  DECIMAL,
    passing_yards_per_scramble                         DECIMAL
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [45]:
insert_statement = f"""
INSERT INTO team_weekly_advanced_stats_running (game_id, team_id, team_name, opponent_id, opponent_name, home_or_away, 
gametime_year, gametime_week, gametime_day, pass_intended_air_yards, passing_intended_air_yards_per_pass_attempt, 
passing_completed_air_yards, passing_completed_air_yards_per_pass_completion, 
passing_completed_air_yards_per_pass_attempt, passing_yards_after_catch, 
passing_yards_after_catch_per_pass_completion, passing_coverage_avg_depth_of_target,
passing_coverage_air_yards_allowed, passing_coverage_yards_after_catch_allowed, pass_rush_blitz, pass_rush_bats,
pass_rush_qb_hurries, pass_rush_qb_knockdowns, pass_rush_pressures, team_quarter_one_points_scored,
team_quarter_two_points_scored, team_quarter_three_points_scored, team_quarter_four_points_scored,
opponents_quarter_one_points_scored, opponents_quarter_two_points_scored, opponents_quarter_three_points_scored,
opponents_quarter_four_points_scored, run_pass_option_plays_run, run_pass_option_yards,
run_pass_option_pass_attempts, run_pass_option_pass_yards, run_pass_option_rush_attempts, 
run_pass_option_rush_yards, play_action_pass_attempts, play_action_pass_yards, passing_batted_passes,
passing_throw_aways, passing_spikes, passing_drops, passing_drop_percentage, passing_poor_throw,
passing_poor_throw_percentage, passing_on_target_pass, passing_on_target_pass_percentage, 
rushing_yards_before_contact, rushing_yards_before_contact_per_rush_attempt, rushing_yards_after_contact,
rushing_yards_after_contact_per_rush_attempt, rushing_broken_tackle, rushing_rushes_per_broken_tackle,
receiving_yards_after_catch, receiving_avg_depth_of_target, receiving_broken_tackle, 
receiving_receptions_per_broken_tackle, receiving_drop, receiving_drop_percentage, passing_avg_pocket_time,
passing_times_blitzed, passing_times_hurried, passing_times_qb_hit, passing_times_qb_pressured,
passing_qb_pressured_percentage, passing_scrambles, passing_yards_per_scramble)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in team_weekly_advanced_stats_running_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [46]:
select_statement = f"""
SELECT *
FROM team_weekly_advanced_stats_running;
"""

sql_team_weekly_advanced_stats_running_df = pd.read_sql_query(select_statement, con = connection)
sql_team_weekly_advanced_stats_running_df.head(3)

,game_id,team_id,team_name,opponent_id,opponent_name,home_or_away,gametime_year,gametime_week,gametime_day,pass_intended_air_yards,...,receiving_drop,receiving_drop_percentage,passing_avg_pocket_time,passing_times_blitzed,passing_times_hurried,passing_times_qb_hit,passing_times_qb_pressured,passing_qb_pressured_percentage,passing_scrambles,passing_yards_per_scramble
0,2021_05_NO_WAS,3300,New Orleans Saints,5110,Washington Commanders,away,2021,5,2021-10-10,176.000000,...,1.250000,5.850000,2.525000,8.000000,2.250000,3.000000,7.000000,25.350000,2.250000,NaN
1,2020_17_PIT_CLE,1050,Cleveland Browns,3900,Pittsburgh Steelers,home,2020,17,2021-01-03,265.333333,...,1.333333,4.400000,2.686667,11.533333,3.133333,2.133333,6.733333,19.306667,1.400000,NaN
2,2022_10_SEA_TB,4900,Tampa Bay Buccaneers,4600,Seattle Seahawks,home,2022,10,2022-11-13,301.222222,...,2.555556,5.944444,2.144444,8.333333,2.000000,2.555556,6.111111,13.688889,0.222222,NaN


## Injuries Table:

In [47]:
depth_chart_df = nfl.import_depth_charts(years)
injuries_df = nfl.import_injuries(years)

combined_df = pd.merge(depth_chart_df, injuries_df, left_on = ['season','week','gsis_id'], right_on = ['season','week','gsis_id'], how = 'left')

combined_df['club_code'] = combined_df['club_code'].replace(to_replace='STL', value='LA')
combined_df['club_code'] = combined_df['club_code'].replace(to_replace='SD', value='LAC')
combined_df['club_code'] = combined_df['club_code'].replace(to_replace='OAK', value='LV')

combined_df = pd.merge(combined_df, team_df, left_on = ['club_code'], right_on = ['team_abbr'], how = 'left')
combined_df = pd.merge(combined_df, sql_game_df, left_on = ['season','week','team_id'], right_on = ['gametime_year','gametime_week','home_team_id'], how = 'left')
combined_df = pd.merge(combined_df, sql_game_df, left_on = ['season','week','team_id'], right_on = ['gametime_year','gametime_week','away_team_id'], how = 'left')

combined_df['game_id'] = combined_df[['game_id_x', 'game_id_y']].bfill(axis=1).iloc[:, 0]

combined_df.loc[combined_df['home_team_id_x'] > 0, 'home_or_away'] = 'Home'
combined_df.loc[combined_df['home_team_id_y'] > 0, 'home_or_away'] = 'Away'

combined_df = combined_df[[
    'game_id', 'team_id', 'home_or_away', 'full_name_x', 'depth_team', 'depth_position', 
    'formation', 'practice_status', 'report_status'
]]

combined_df = combined_df.dropna(axis=0, subset=['game_id'])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'game_type_x'} in the result is deprecated and will raise a MergeError in a future version.
  if sys.path[0] == '':


In [48]:
create_statement = f"""
DROP TABLE IF EXISTS injuries;

CREATE TABLE injuries (
    game_id                                 VARCHAR(50) NOT NULL,
    team_id                                 INT,
    home_or_away                            VARCHAR(30),
    player_name                             VARCHAR(200),
    depth_team                              INT,
    depth_position                          VARCHAR(50),
    formation                               VARCHAR(200),
    practice_status                         VARCHAR(200),
    report_status                           VARCHAR(200)
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [49]:
insert_statement = f"""
INSERT INTO injuries (game_id, team_id, home_or_away, player_name, depth_team, depth_position, 
formation, practice_status, report_status)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in combined_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [50]:
select_statement = f"""
SELECT *
FROM injuries;
"""

injuries_sql_df = pd.read_sql_query(select_statement, con = connection)
injuries_sql_df.head(3)

,game_id,team_id,home_or_away,player_name,depth_team,depth_position,formation,practice_status,report_status
0,2015_13_ATL_TB,200,Away,Shayne Graham,1,K,Special Teams,NaN,NaN
1,2015_14_ATL_CAR,200,Away,Shayne Graham,1,K,Special Teams,NaN,NaN
2,2015_15_ATL_JAX,200,Away,Shayne Graham,1,K,Special Teams,NaN,NaN


## Weekly Injuries Table:

In [51]:
select_statement = f"""
SELECT game_id,
    team_id,
    home_or_away,
    
    SUM(CASE WHEN formation = 'Offense' AND report_status = 'Out' THEN 1 ELSE 0 END) AS offense_starters_out,
    SUM(CASE WHEN formation = 'Offense' AND report_status = 'Doubtful' THEN 1 ELSE 0 END) AS offense_starters_doubtful,
    SUM(CASE WHEN formation = 'Offense' AND report_status = 'Questionable' THEN 1 ELSE 0 END) AS offense_starters_questionable,

    SUM(CASE WHEN formation = 'Defense' AND report_status = 'Out' THEN 1 ELSE 0 END) AS defense_starters_out,
    SUM(CASE WHEN formation = 'Defense' AND report_status = 'Doubtful' THEN 1 ELSE 0 END) AS defense_starters_doubtful,
    SUM(CASE WHEN formation = 'Defense' AND report_status = 'Questionable' THEN 1 ELSE 0 END) AS defense_starters_questionable,
    
    SUM(CASE WHEN formation = 'Special Teams' AND report_status = 'Out' THEN 1 ELSE 0 END) AS special_starters_out,
    SUM(CASE WHEN formation = 'Special Teams' AND report_status = 'Doubtful' THEN 1 ELSE 0 END) AS special_starters_doubtful,
    SUM(CASE WHEN formation = 'Special Teams' AND report_status = 'Questionable' THEN 1 ELSE 0 END) AS special_starters_questionable
    
FROM injuries
WHERE depth_team = 1
GROUP BY 1, 2, 3
"""

weekly_injuries_df = pd.read_sql_query(select_statement, con = connection)

In [52]:
create_statement = f"""
DROP TABLE IF EXISTS weekly_injuries;

CREATE TABLE weekly_injuries (
    game_id                                 VARCHAR(50) NOT NULL,
    team_id                                 INT,
    home_or_away                            VARCHAR(30),
    offense_starters_out                    INT,
    offense_starters_doubtful               INT,
    offense_starters_questionable           INT,
    defense_starters_out                    INT,
    defense_starters_doubtful               INT,
    defense_starters_questionable           INT,
    special_starters_out                    INT,
    special_starters_doubtful               INT,
    special_starters_questionable           INT
);
"""

with connection, connection.cursor() as cursor:
    cursor.execute(create_statement)

In [53]:
insert_statement = f"""
INSERT INTO weekly_injuries (game_id, team_id, home_or_away, offense_starters_out, offense_starters_doubtful, 
offense_starters_questionable, defense_starters_out, defense_starters_doubtful, defense_starters_questionable, 
special_starters_out, special_starters_doubtful, special_starters_questionable)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with connection, connection.cursor() as cursor:
    for row in weekly_injuries_df.itertuples(index = False, name = None):
        cursor.execute(insert_statement, row)

In [54]:
select_statement = f"""
SELECT *
FROM weekly_injuries;
"""

weekly_injuries_sql_df = pd.read_sql_query(select_statement, con = connection)
weekly_injuries_sql_df.head(3)

,game_id,team_id,home_or_away,offense_starters_out,offense_starters_doubtful,offense_starters_questionable,defense_starters_out,defense_starters_doubtful,defense_starters_questionable,special_starters_out,special_starters_doubtful,special_starters_questionable
0,2015_01_BAL_DEN,325,Away,0,0,0,0,1,0,0,0,0
1,2015_01_BAL_DEN,1400,Home,0,0,0,0,0,0,0,0,0
2,2015_01_CAR_JAX,750,Away,0,0,0,1,0,0,0,0,0


## Grant Permissions on Tables:

In [55]:
permissions_statement = f"""
    GRANT ALL PRIVILEGES ON game TO ajmvgd;
    GRANT ALL PRIVILEGES ON game TO cad7v7;
    GRANT ALL PRIVILEGES ON game TO txdy8;
    GRANT ALL PRIVILEGES ON game TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON team TO ajmvgd;
    GRANT ALL PRIVILEGES ON team TO cad7v7;
    GRANT ALL PRIVILEGES ON team TO txdy8;
    GRANT ALL PRIVILEGES ON team TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON line TO ajmvgd;
    GRANT ALL PRIVILEGES ON line TO cad7v7;
    GRANT ALL PRIVILEGES ON line TO txdy8;
    GRANT ALL PRIVILEGES ON line TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON player_weekly_stats TO ajmvgd;
    GRANT ALL PRIVILEGES ON player_weekly_stats TO cad7v7;
    GRANT ALL PRIVILEGES ON player_weekly_stats TO txdy8;
    GRANT ALL PRIVILEGES ON player_weekly_stats TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON player_yearly_stats TO ajmvgd;
    GRANT ALL PRIVILEGES ON player_yearly_stats TO cad7v7;
    GRANT ALL PRIVILEGES ON player_yearly_stats TO txdy8;
    GRANT ALL PRIVILEGES ON player_yearly_stats TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON team_weekly_stats TO ajmvgd;
    GRANT ALL PRIVILEGES ON team_weekly_stats TO cad7v7;
    GRANT ALL PRIVILEGES ON team_weekly_stats TO txdy8;
    GRANT ALL PRIVILEGES ON team_weekly_stats TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON team_weekly_stats_running TO ajmvgd;
    GRANT ALL PRIVILEGES ON team_weekly_stats_running TO cad7v7;
    GRANT ALL PRIVILEGES ON team_weekly_stats_running TO txdy8;
    GRANT ALL PRIVILEGES ON team_weekly_stats_running TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON team_weekly_advanced_stats TO ajmvgd;
    GRANT ALL PRIVILEGES ON team_weekly_advanced_stats TO cad7v7;
    GRANT ALL PRIVILEGES ON team_weekly_advanced_stats TO txdy8;
    GRANT ALL PRIVILEGES ON team_weekly_advanced_stats TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON team_weekly_advanced_stats_running TO ajmvgd;
    GRANT ALL PRIVILEGES ON team_weekly_advanced_stats_running TO cad7v7;
    GRANT ALL PRIVILEGES ON team_weekly_advanced_stats_running TO txdy8;
    GRANT ALL PRIVILEGES ON team_weekly_advanced_stats_running TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON injuries TO ajmvgd;
    GRANT ALL PRIVILEGES ON injuries TO cad7v7;
    GRANT ALL PRIVILEGES ON injuries TO txdy8;
    GRANT ALL PRIVILEGES ON injuries TO mtrkkf;
    
    GRANT ALL PRIVILEGES ON weekly_injuries TO ajmvgd;
    GRANT ALL PRIVILEGES ON weekly_injuries TO cad7v7;
    GRANT ALL PRIVILEGES ON weekly_injuries TO txdy8;
    GRANT ALL PRIVILEGES ON weekly_injuries TO mtrkkf;
"""

with connection, connection.cursor() as cursor:
    cursor.execute(permissions_statement)

## Close Database Connection:

In [56]:
connection.close()

In [57]:
connection

<connection object at 0x7f9c8b122a08; dsn: 'user=mtrkkf password=xxx dbname=capstonesp23olt05 host=pgsql.dsa.lan', closed: 1>